In [13]:
# the aim is to end up with train, test, evaluate
import sys
sys.path.append("/Users/david/Documents/code/fhmm/v1")

In [24]:
import src.ads_davidjames9610.ads as ads
import src.ads_davidjames9610.useful as useful
import numpy as np
import os
import os.path
from pathlib import Path
import matplotlib.pyplot as plt
from importlib import reload
import importlib
import src.classifiers_davidjames9610.test_a.config as base_config
reload(base_config)
config = importlib.import_module(base_config.config_location)
reload(config)
print(config.basedir)
import src.misc_davidjames9610.fe_methods as fe
import src.misc_davidjames9610.proc_methods as pm

/Users/david/Documents/code/fhmm/v1/src/classifiers_davidjames9610/test_a/speech_commands


"""
we want to end up with this

    section 2, samples to features,
    -- choose fe_method,
    -- noise and reverb, apply to test / evaluate before fe method,

    select fe_methods,
    -- this point also is important for choosing cross-validation, and train-test-eval split
    - fe_methods = ['mfcc', 'log-power', 'lfcc']
    - select noise or reverb to be applied to eval and test
    - post_proc = ['noise', 'reverb']

    -- labels don't change

    features = {
        'mfcc':
            'train': [],
            'train_labels': [cv_index_0] [cv_index_1]
            'test': [],
            'test_labels': []   // test and eval will need different process method if noise is being added
            'eval': [],
            'eval_labels': []
        'lfcc':
            ...
    }
"""

In [25]:
import pickle
sls = pickle.load(open(config.samples_labels,'rb'))

In [26]:
# going to have to do fe after cross-validation split
import src.misc_davidjames9610.cross_validation as cv
from src.misc_davidjames9610.cross_validation import *
import importlib
importlib.reload(cv)
import b_samples_to_features
reload(b_samples_to_features)
import src.misc_davidjames9610.noisey as noisey
reload(noisey)
import random

# define config things
n_folds = 2
cv_output = cv.split_data_for_cross_validation(sls['samples'], sls['labels'], n_folds) # , test_ratio=0.8) # could update test ratio to
signal_average_power = b_samples_to_features.get_average_power_for_samples(cv_output)
signal_average_power_db = 10 * np.log10(signal_average_power)
max_sample_length = np.max([len(sample) for sample in sls['samples']])


In [27]:
from src.misc_davidjames9610.noisey import get_noise_avg_watts
import importlib
importlib.reload(fe)
nfft = 128

class ProcessingBase:
    def __init__(self, fe_method):
        self.fe_method = fe_method
    def __str__(self):
        return self.fe_method.__str__()
    def pre_process(self, sample):
        return self.fe_method(sample)
    def post_process(self, sample):
        return self.fe_method(sample)
    def get_noise_feature(self):
        return []

class ProcessingGaussNoise:
    def __init__(self, fe_method, noise_snr, signal_power, sample_len):
        self.fe_method = fe_method
        self.noise_snr = noise_snr
        self.signal_power = signal_power
        self.noise_sample = noisey.get_noise_for_sample(sample_len, self.signal_power,self.noise_snr, self.noise_snr, self.noise_snr)

    def __str__(self):
        return self.fe_method.__str__() + 'GaussNoise_SNR' + str(self.noise_snr)
    def pre_process(self, sample):
        return self.fe_method(sample)
    def post_process(self, sample):
        sample = self.noise_sample[:len(sample)] + sample
        return self.fe_method(sample)
    def get_noise_feature(self):
        return self.fe_method(self.noise_sample)



class ProcessingRealNoise:
    def __init__(self, fe_method, noise_snr, signal_power, sample_len, noise_key):

        self.fe_method = fe_method
        self.noise_snr = noise_snr
        self.signal_power = signal_power



        self.noise_sample = noisey.get_noise_for_sample(sample_len, self.signal_power,self.noise_snr, self.noise_snr, self.noise_snr)

    def __str__(self):
        return self.fe_method.__str__() + 'GaussNoise_SNR' + str(self.noise_snr)
    def pre_process(self, sample):
        return self.fe_method(sample)
    def post_process(self, sample):
        sample = self.noise_sample[:len(sample)] + sample
        return self.fe_method(sample)
    def get_noise_feature(self):
        return self.fe_method(self.noise_sample)


class ProcessingGaussNoiseReverb:
    def __init__(self, fe_method, noise_snr, signal_power, sample_len):
        self.noise_snr = noise_snr
        self.signal_power = signal_power
        self.fe_method = fe_method
    def __str__(self):
        return self.fe_method.__str__() + 'GaussNoise_SNR' + str(self.noise_snr) + '_Reverb'
    def pre_process(self, sample):
        return self.fe_method(sample)
    def post_process(self, sample):
        # todo update so Gauss Noise Reverb works
        return self.fe_method(sample)
    def get_noise_feature(self):
        # update
        return []


In [ ]:
import numpy as np

def scale_signal_to_db(signal, current_db, target_db):
    # Convert current dB level to power in watts
    current_power_watts = 10 ** (current_db / 10)

    # Convert target dB level to power in watts
    target_power_watts = 10 ** (target_db / 10)

    # Calculate the scaling factor
    scaling_factor = np.sqrt(target_power_watts / current_power_watts)

    # Scale the signal
    scaled_signal = signal * scaling_factor

    return scaled_signal


In [35]:

def get_real_noise_sample(noise_key, target_snr_db, signal_power, sample_len):
    # 1. read in noise sample and normalise
    noise_sample = useful.file_to_audio(noise_key, config.sr)[0]

    if len(noise_sample) < sample_len:
        print('oh no; len(noise_sample) < sample_len')

    # 2. scale the power
    noise_avg_watts = np.mean(noise_sample ** 2)

    print('original noise watts', noise_avg_watts)

    noise_target_db = signal_power - target_snr_db
    noise_target_watts = 10 ** (noise_target_db / 10)

    # Calculate the scaling factor
    scaling_factor = np.sqrt(noise_target_watts / noise_avg_watts)
    scaled_noise_sample = noise_sample * scaling_factor

    # print watts
    print('signal watts', 10 ** (signal_power / 10))
    print('noise watts', np.mean(scaled_noise_sample ** 2))

    # db
    print('signal dbs', signal_power)
    print('noise dbs', 10 * np.log10(np.mean(scaled_noise_sample ** 2)))
    print('target dbs', target_snr_db)

    return scaled_noise_sample

get_real_noise_sample(base_config.UNDER_WATER_DRONE, 20, signal_average_power, max_sample_length)


original noise watts 0.09872509
signal watts 1.0263409085549056
noise watts 0.0102634085
signal dbs 0.11291639638021203
noise dbs -19.887083768844604
target dbs 20


array([-9.8397059e-06, -9.8397059e-06,  0.0000000e+00, ...,
        6.3960010e-04,  5.7072216e-04,  3.5424862e-04], dtype=float32)

In [30]:
import librosa
reload(base_config)
librosa.load(base_config.UNDER_WATER_DRONE)

(array([-3.2449909e-05, -3.7455582e-05, -1.1038762e-05, ...,
         1.7091444e-03,  1.1040343e-03,  0.0000000e+00], dtype=float32),
 22050)

In [ ]:

proc_methods = [
    # ProcessingBase(
    #     fe_method=fe.FeatureExtractorLogPower(nfft=128)),
    ProcessingBase(
        fe_method=fe.FeatureExtractorLogPower(nfft=128)),
    ProcessingGaussNoise(
        fe_method=fe.FeatureExtractorLogPower(nfft=128),
        noise_snr=60,
        signal_power=signal_average_power,
        sample_len=max_sample_length),
    ProcessingGaussNoise(
        fe_method=fe.FeatureExtractorLogPower(nfft=128),
        noise_snr=40,
        signal_power=signal_average_power,
        sample_len=max_sample_length),
    ProcessingGaussNoise(
        fe_method=fe.FeatureExtractorLogPower(nfft=128),
        noise_snr=20,
        signal_power=signal_average_power,
        sample_len=max_sample_length),
    ProcessingGaussNoise(
        fe_method=fe.FeatureExtractorLogPower(nfft=128),
        noise_snr=10,
        signal_power=signal_average_power,
        sample_len=max_sample_length),
    # ProcessingBase(
    #     fe_method=fe.FeatureExtractorMfcc(nfft=128, num_ceps=8, fs=config.sr)),
    # ProcessingGaussNoise(
    #     fe_method=fe.FeatureExtractorMfcc(nfft=128, num_ceps=8, fs=config.sr),
    #     noise_snr=60,
    #     signal_power=signal_average_power,
    #     sample_len=max_sample_length),
    #     ProcessingGaussNoise(
    #     fe_method=fe.FeatureExtractorMfcc(nfft=128, num_ceps=8, fs=config.sr),
    #     noise_snr=40,
    #     signal_power=signal_average_power,
    #     sample_len=max_sample_length),
    #     ProcessingGaussNoise(
    #     fe_method=fe.FeatureExtractorMfcc(nfft=128, num_ceps=8, fs=config.sr),
    #     noise_snr=20,
    #     signal_power=signal_average_power,
    #     sample_len=max_sample_length),
    #     ProcessingGaussNoise(
    #     fe_method=fe.FeatureExtractorMfcc(nfft=128, num_ceps=8, fs=config.sr),
    #     noise_snr=10,
    #     signal_power=signal_average_power,
    #     sample_len=max_sample_length),
    ]

for proc_method in proc_methods:
    print(proc_method.__str__())


In [21]:
reload(fe)

features = {
}

import b_samples_to_features
reload(b_samples_to_features)

for proc_method in proc_methods:

    fe_name = proc_method.__str__()
    features[fe_name] = {}
    print('processing for: ', fe_name)
    n_folds = len(cv_output['train_data'])

    # train
    train_features = []
    train_labels = []
    for i in range(n_folds):
        curr_features = [proc_method.pre_process(sample) for sample in cv_output['train_data'][i]]
        curr_labels = cv_output['train_label'][i]
        train_features.append(curr_features)
        train_labels.append(curr_labels)
    features[fe_name]['train_features'] = train_features
    features[fe_name]['train_label'] = train_labels

    # val
    val_features = []
    val_labels = []
    val_noise = []
    for i in range(n_folds):
        curr_features = []
        curr_noise_features = []

        for sample in cv_output['val_data'][i]:
            curr_feature = proc_method.post_process(sample)
            curr_features.append(curr_feature)

        curr_noise_features.append(proc_method.get_noise_feature())

        curr_labels = cv_output['val_label'][i]
        val_features.append(curr_features)
        val_labels.append(curr_labels)
        val_noise.append(curr_noise_features)

    features[fe_name]['val_features'] = val_features
    features[fe_name]['val_label'] = cv_output['val_label']
    features[fe_name]['noise_features'] = val_noise


processing for:  lp
processing for:  lpGaussNoise_SNR60
processing for:  lpGaussNoise_SNR40
processing for:  lpGaussNoise_SNR20
processing for:  lpGaussNoise_SNR10


In [22]:
import pickle
pickle.dump(features, open(config.features,'wb'))

In [23]:
config.features

'/Users/david/Documents/code/fhmm/v1/src/classifiers_davidjames9610/test_a/speech_commands/features.pickle'